In [1]:
import numpy as np
import scipy.linalg
import cvxpy as cp
from scipy.linalg import block_diag, solve_discrete_are

Ref: Distributionally robust control of constrained stochastic systems

In [2]:
def dlqr(A,B,Q,R):
    """
    Calculates the optimal gain matrix K for system
    x[k+1] = A x[k] + B u[k]
    with cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k], such that
    u[k] = -K x[k]
    
    Returns: 
        -K
    """
    # first, solve the ricatti equation
    P = np.matrix(scipy.linalg.solve_discrete_are(A, B, Q, R))
    # compute the LQR gain
    K = np.matrix(scipy.linalg.inv(B.T*P*B+R)*(B.T*P*A))
    return -K

In [3]:
def disc_linear_system(A,B,delta_t):
    '''
    Discrete a linear system with implicit Euler
    x[k+1] = (I - delta_t * A)^{-1} @ x[k] + (I - delta_t * A)^{-1} @ (delta_t * B) @ u[k]
    
    Returns:
        Ak
        Bk
    
    ''' 
    Nx = np.shape(A)[0]
    Ix = np.identity(Nx)
    
    Ak = np.linalg.inv(Ix - delta_t * A)
    Bk = np.linalg.inv(Ix - delta_t * A) @ (delta_t * B)
    
    
    return Ak, Bk

In [4]:
def RK4_np(f, x, u, t, h):
    """
    Runge-Kutta 4th order solver using numpy array data type.

    Args:
        f: A function returning first order ODE in 2D numpy array (Nx x 1).
        x: Current value (list or numpy array). 
        t: Current time.
        h: Step length.
    Returns:
        x_next: Vector of next value in 2D numpy array (Nx x 1)
    """
    x = np.reshape(x, (np.shape(x)[0], -1))    # Reshape x to col vector in np 2D array
    k1 = f(t, x, u)
    k2 = f(t + h / 2, x + h / 2 * k1, u)
    k3 = f(t + h / 2, x + h / 2 * k2, u)
    k4 = f(t + h, x + h * k3, u)
    x_next = x + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
    return x_next

In [5]:
def mass_string_ode(t, x, u):
    m = 2 #[kg]
    k1 = 3 # [N/m]
    k2 = 2 # [N/m]
    
    A = np.array([[0,1],[-k2/m, -k1/m]])
    B = np.array([[0],[1/m]])
    
    dot_x = A @ x + B @ u
    
    return dot_x

Continuous-time system

$\begin{aligned}\left[\begin{array}{c}\dot{x}_{1}(t) \\ \dot{x}_{2}(t)\end{array}\right] &=\left[\begin{array}{cc}0 & 1 \\ -k_{2} / m & -k_{1} / m\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right]+\left[\begin{array}{c}0 \\ 1 / m\end{array}\right] u(t) \\ y(t) &=\left[\begin{array}{ll}1 & 0\end{array}\right]\left[\begin{array}{l}x_{1}(t) \\ x_{2}(t)\end{array}\right] \end{aligned}$

In [6]:
m = 2 #[kg]
k1 = 3 # [N/m]
k2 = 2 # [N/m]

para = [m, k1, k2]

In [7]:
A = np.array([[0,1],[-k2/m, -k1/m]])
B = np.array([[0],[1/m]])
delta_t = 0.1

In [8]:
Ak,Bk = disc_linear_system(A,B,delta_t)
Ck = np.array([[0, 0],[1, 0]])

In [9]:
Dk = np.array([[1,0]])
Ek = np.array([[0, 1]])

In [10]:
Q = np.diag([10,1])
R = np.diag([1])

In [11]:
neg_K = dlqr(Ak,Bk,Q,R)

In [12]:
neg_K

matrix([[-1.64465128, -1.02359576]])

In [13]:
N_sim = 100

x_init = np.array([[2],[0]])
t0 = 0

xk = x_init
uk = 0
t = t0
h = delta_t

x_list = []
x_list += [xk.flatten().tolist()]
u_list = []
for i in range(N_sim):
    uk = neg_K @ xk
    u_list +=  uk.flatten().tolist() 
    x_kp1 = RK4_np(mass_string_ode, xk, uk, t, h)
    x_list += x_kp1.flatten().tolist()
    xk = x_kp1

In [14]:
u_list

[[-3.289302564456352],
 [-2.914942510901101],
 [-2.559811018238362],
 [-2.2263935494494858],
 [-1.9163535315592184],
 [-1.6306546787177174],
 [-1.369672758698418],
 [-1.1332968450937406],
 [-0.9210202675311228],
 [-0.7320216075616044],
 [-0.5652361925567164],
 [-0.41941861782554024],
 [-0.2931968818221259],
 [-0.1851187540790109],
 [-0.0936910134284134],
 [-0.017412197937609127],
 [0.045200499702823194],
 [0.09558957459547417],
 [0.1351401974285432],
 [0.16516728509624534],
 [0.18690612552290364],
 [0.20150606471958082],
 [0.21002680236539473],
 [0.21343688113744064],
 [0.2126139938328937],
 [0.2083467703985401],
 [0.20133774379984534],
 [0.19220722883086472],
 [0.18149788119473245],
 [0.16967973526549882],
 [0.1571555477432043],
 [0.14426630086558034],
 [0.1312967429244227],
 [0.11848086557844487],
 [0.10600723691806505],
 [0.09402412650932193],
 [0.08264437383307388],
 [0.07194996476618828],
 [0.06199629215829093],
 [0.052816086281720964],
 [0.04442300911719047],
 [0.0368149132265959

In [15]:
x_list

[[2, 0],
 [1.9826689238506545, -0.337883851188942],
 [1.9349862198417493, -0.6082152491804027],
 [1.8633130681783237, -0.8187867821451399],
 [1.7732520651954105, -0.9769752784440978],
 [1.669691328807902, -1.089693169266517],
 [1.5568526124067068, -1.163356605030776],
 [1.4383418800523347, -1.2038677983761676],
 [1.3172010373594736, -1.2166092885486843],
 [1.1959597318770925, -1.2064480423099309],
 [1.0766863343887605, -1.1777475217519346],
 [0.9610373890339324, -1.134386056870654],
 [0.8503049763687974, -1.0797800583539778],
 [0.7454615705111942, -1.0169107922407186],
 [0.6472020905436895, -0.9483536118440088],
 [0.5559829486632923, -0.8763087029491551],
 [0.4720579844946872, -0.8026325454778964],
 [0.3955112478724361, -0.7288694285283872],
 [0.3262866525834703, -0.6562824761564078],
 [0.2642145723515755, -0.5858837488486628],
 [0.20903548900109586, -0.5184630808900762],
 [0.16042083245903396, -0.454615397390631],
 [0.11799117417557238, -0.3947663273396302],
 [0.08133195071989681, -0.

FINITE HORIZON DISTRIBUTIONALLY ROBUST CONTROL PROBLEMS

inf $\quad \tilde{J}_{N}(x, U)$

s.t.
$U \in \mathcal{N}, \beta_{t} \in \mathbb{R}, X_{t} \in \mathbb{S}_{+}^{N d+2}, P_{t}^{i} \in \mathbb{S}_{+}^{N d+1}$

$\beta_{t}+\frac{1}{\epsilon} \operatorname{Tr}\left\{M_{w} X_{t}\right\} \leq 0,$

$\quad X_{t}-\left(\begin{array}{cc}\alpha_{i} P_{t}^{i} & \alpha_{i}\left(\mathcal{B}_{t} U+\mathcal{C}_{t}\right)^{\top} e_{i} \\ e_{i}^{\top}\left(\mathcal{B}_{t} U+\mathcal{C}_{t}\right) \alpha_{i} & \alpha_{i} e_{i}^{0}-\beta_{t}\end{array}\right) \succeq 0,$

$\left(\begin{array}{cc}P_{t}^{i} & \left(\mathcal{B}_{t} U+\mathcal{C}_{t}\right)^{\top} E_{i}^{1 / 2} \\ E_{i}^{1 / 2}\left(\mathcal{B}_{t} U+\mathcal{C}_{t}\right) & \mathbb{I}_{n}\end{array}\right) \succeq 0,$

Cost function:
$\tilde{J}_{N}(x, U):=\operatorname{Tr}\left\{U^{\top}\left(J_{u}+\mathcal{B} J_{x} \mathcal{B}\right) U M_{w}+2 \mathcal{C} J_{x} \mathcal{B} U M_{w}+\mathcal{C}^{\top} J_{x} \mathcal{C} M_{w}\right\}$

In [16]:
n = 2
m = 1
d = 2
r = 1
N = 4

Nx = (N + 1) * n
Nu = N * m
Nw = N*d + 1
Ny = r * N

In [17]:
mu = np.array([[0],[0]])

$\mathcal{B} \in \mathbb{R}^{N_{x} \times N_{u}}, \mathcal{C} \in \mathbb{R}^{N_{x} \times N_{w}}, \mathcal{D} \in \mathbb{R}^{N_{y} \times N_{u}}$ and $\mathcal{E} \in \mathbb{R}^{N_{y} \times N_{u}}$

In [18]:
U = cp.Variable((Nu,Nw))
beta_disc = 0.5 # discount variable
beta_list = [beta_disc ** i for i in range(N + 1)] # discount variable

Q = np.diag([10, 1] )
Qf = np.diag([10, 1])
kron1 = np.kron(np.diag(beta_list[0:-1]), Q)
Jx = block_diag(kron1, beta_list[-1] * Qf)

R = np.diag([1])
Ju = np.kron(np.diag(beta_list[0:-1]), R)

In [19]:
B_ext = np.zeros([Nx,Nu])

for i in range(N):
    ANB = Bk
    for j in range(i):
        if i != 0:
            ANB = Ak @ ANB
    for k in range(N-i):
        B_ext[(1 + i + k) * n : (1 + i + 1 + k) * n , (0 + k) * m : (1+k) * m] = ANB

In [20]:
B_ext

array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.00431034, 0.        , 0.        , 0.        ],
       [0.04310345, 0.        , 0.        , 0.        ],
       [0.007989  , 0.00431034, 0.        , 0.        ],
       [0.03678656, 0.04310345, 0.        , 0.        ],
       [0.01109139, 0.007989  , 0.00431034, 0.        ],
       [0.03102385, 0.03678656, 0.04310345, 0.        ],
       [0.01367024, 0.01109139, 0.007989  , 0.00431034],
       [0.02578854, 0.03102385, 0.03678656, 0.04310345]])

In [21]:
C_ext = np.zeros([Nx,Nw - 1])

for i in range(N):
    ANC = Ck
    for j in range(i):
        if i != 0:
            ANC = Ak @ ANC
    for k in range(N-i):
        C_ext[(1 + i + k) * n : (1 + i + 1 + k) * n , (0 + k) * d : (1+k) * d] = ANC

C_ANx0_col = np.zeros([Nx, 1])
ANx0 = x_init
for i in range(N + 1):
    if i != 0:
        ANx0 = Ak @ ANx0
    C_ANx0_col[i * n : (i+1) * n, :] = ANx0
C_ext = np.hstack((C_ANx0_col,C_ext))

In [22]:
C_ext

array([[ 2.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.98275862,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.17241379,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.95080262,  0.0862069 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.31956005,  0.86206897,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.90643707,  0.15978002,  0.        ,  0.0862069 ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.44365544,  0.73573127,  0.        ,  0.86206897,  0

In [23]:
C_ANx0_col

array([[ 2.        ],
       [ 0.        ],
       [ 1.98275862],
       [-0.17241379],
       [ 1.95080262],
       [-0.31956005],
       [ 1.90643707],
       [-0.44365544],
       [ 1.85175611],
       [-0.54680961]])

In [24]:
Dk

array([[1, 0]])

In [25]:
blocks = np.multiply.outer(np.arange(1,N), Dk)
blocks
# aux

array([[[1, 0]],

       [[2, 0]],

       [[3, 0]]])

In [26]:
Nx

10

In [27]:
D_ext = np.zeros([Ny, Nu])

blocks = [Dk] * (N)
offset = r
aux = np.empty((0, offset), int)
D_ext = block_diag(aux.T, *blocks, aux)
D_ext

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0]])

In [28]:
E_ext = np.zeros([Ny, Nw])

blocks = [1] + [Ek] * (N)
E_ext = block_diag(*blocks)

In [29]:
Ek

array([[0, 1]])

In [30]:
E_ext

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [31]:
mu_list = mu.flatten().tolist()

Mw_row = np.array([[1] + mu_list * N])

Mw = Mw_row.T @ Mw_row

In [32]:
Mw_row
Mw

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [33]:
Jx.shape

(10, 10)

In [34]:
C_ext.shape

(10, 9)

In [35]:
objective = cp.Minimize(cp.trace(U.T @ (Ju + B_ext.T @ Jx @ B_ext) @ U @ Mw + 2 * C_ext.T @ Jx @ B_ext @ U @ Mw + C_ext.T @ Jx @ C_ext @ Mw))

In [36]:
# # Causality set
# # u_var_list = [cp.Variable((m , 1)) for i in range(N)]
# u_var = cp.Variable((Nu , 1))

# rhs_matrix = np.zeros([Nu,Ny])

# for i in rangeM:
#     for k in range(i, N):
#         print(rhs_matrix[(k) * m : (k + 1) * m, (i) * d : ( i +1 ) * d].shape)
#         print(U[ k * m : ( k+1 ) * m, (i-1) * d : i * d].shape)
#         rhs_matrix[(k) * m : (k + 1) * m, (i) * d : ( i +1 ) * d] = U[ k * m : ( k+1 ) * m, (i-1) * d : i * d]

In [37]:
# Causality set
# u_var_list = [cp.Variable((m , 1)) for i in range(N)]
u_var = cp.Variable((Nu , 1))

rhs_matrix = u_var

for i in range(N):
    col = np.zeros((m , r))
    for j in range(1, N):
        # if i == (N-1):
        #     if j >= i:
        #         col = cp.vstack([col, U[j * m: ( j+1 ) * m, i * r : (i+1) * r]])
        #     else:
        #         col = cp.vstack([col, np.zeros((m, r))])
        # else:
        if j > i:
            col = cp.vstack([col, U[j * m: ( j+1 ) * m, i * r : (i+1) * r]])
        else:
            col = cp.vstack([col, np.zeros((m, r))])

    print(col)

    rhs_matrix = cp.hstack([rhs_matrix, col])

print(rhs_matrix.shape)
print(Nu)
print(Ny)
print(d)

Vstack(Vstack(Vstack([[0.]], var0[1, 0]), var0[2, 0]), var0[3, 0])
Vstack(Vstack(Vstack([[0.]], [[0.]]), var0[2, 1]), var0[3, 1])
Vstack(Vstack(Vstack([[0.]], [[0.]]), [[0.]]), var0[3, 2])
Vstack(Vstack(Vstack([[0.]], [[0.]]), [[0.]]), [[0.]])
(4, 5)
4
4
2


In [38]:
# constraint = [U == rhs_matrix @ (D_ext @ C_ext + E_ext)]

In [39]:
# (D_ext @ C_ext + E_ext)

In [40]:
# D_ext.shape


###Infinite horizon version

In [115]:
E0 = np.diag([1,1])
e0 = -50
epsilon = 0.2

In [116]:
A_dare = Ak.T
Q_dare = Ck @ Ck.T
B_dare = Dk.T
R_dare = Ek @ Ek.T

In [117]:
print(Dk)

[[1 0]]


In [118]:
print(A_dare.shape)
print(B_dare.shape)
print(Q_dare.shape)
print(R_dare.shape)

(2, 2)
(2, 1)
(2, 2)
(1, 1)


In [119]:
Y_dare = solve_discrete_are(A_dare, B_dare, Q_dare, R_dare)

In [120]:
Y_dare

array([[0.33531025, 0.52482326],
       [0.52482326, 3.0764769 ]])

In [121]:
# P = cp.Variable((n, n),symmetric=True)
P = cp.Variable((n, n),PSD=True)
# P = cp.Variable((n, n))
Z = cp.Variable((m, n))
# X = cp.Variable((m, m),symmetric=True)
X = cp.Variable((m, m),PSD=True)
# X = cp.Variable((m, m))
W_tilde = Ak@Y_dare@Dk.T  @ np.linalg.inv(Dk@Y_dare@Dk.T +Ek@Ek.T) @Dk@Y_dare@Ak.T

constraint1 = cp.hstack([ cp.vstack([X,Z.T]),cp.vstack([Z,P]) ])
constraint2 = e0 + 1/epsilon * cp.trace(E0 @ (Y_dare +P))
constraint3 = cp.hstack([cp.vstack([ P - Ak@P@Ak.T - Bk@Z@Ak.T - Ak@Z.T@Bk.T - W_tilde, Z.T@Bk.T]), cp.vstack([Bk@Z, P])])

constraints = [constraint1 >> 0]
constraints += [constraint2 <= 0]
constraints += [constraint3 >> 0]


obj =cp.trace(Q @ (Y_dare + P)) + cp.trace( R @ X) 

In [122]:
W_tilde

array([[0.10681365, 0.11978549],
       [0.11978549, 0.13433266]])

In [123]:
prob = cp.Problem(cp.Minimize(obj),
                  constraints)
prob.solve()

27.686362996718024

In [124]:
print(prob.status)

optimal


In [125]:
print(Z.value)

[[-2.30983343 -0.21794331]]


In [126]:
print(P.value)

[[ 1.64120006 -0.37982024]
 [-0.37982024  0.82306844]]


In [127]:
K_opt = Z.value @ np.linalg.inv(P.value)
K_opt

array([[-1.64429116, -1.02358242]])